In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# checking availability of GPU

tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
batch_size = 16

training_datagenerator = ImageDataGenerator(
    rescale= 1./255,
    horizontal_flip=True,
    vertical_flip=True,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.1
    )

In [ ]:
train = training_datagenerator.flow_from_directory(
'/content/gdrive/MyDrive/Takeo/Fire_Smoke/train',
target_size = (224,224),
color_mode='rgb',
class_mode='binary',
batch_size=batch_size,
subset='training'
)

Found 1802 images belonging to 2 classes.


In [ ]:
validation = training_datagenerator.flow_from_directory(
'/content/gdrive/MyDrive/Takeo/Fire_Smoke/validate',
target_size = (224,224),
color_mode='rgb',
class_mode='binary',
batch_size=batch_size,
subset='training'
)

Found 902 images belonging to 2 classes.


In [ ]:
cnn = tf.keras.models.Sequential()

In [ ]:
# adding first layer

cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,padding='same',activation='relu',input_shape=[224,224,3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2))

In [ ]:
# adding second layer

cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3,padding='same',activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2))

In [ ]:
# adding third layer

cnn.add(tf.keras.layers.Conv2D(filters=256,kernel_size=3,padding='same',activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2))

In [ ]:
# flattening

cnn.add(tf.keras.layers.Flatten())

In [ ]:
# Fully connected layer

cnn.add(tf.keras.layers.Dense(units=128,activation='relu'))

In [ ]:
# Output layers

cnn.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

In [ ]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 56, 56, 256)       73984     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 28, 28, 256)      0

In [ ]:
# Compile and Train

checkpoint=tf.keras.callbacks.ModelCheckpoint('/content/gdrive/MyDrive/Fire_Smoke/Fire_and_Smoke.h5',
                                             monitor='val_loss',mode="min",
                                             save_best_only=True)
callbacks=checkpoint

In [ ]:
cnn.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

cnn.fit_generator(train,validation_data=validation,epochs=1,
                  steps_per_epoch=train.samples//batch_size,
                  validation_steps=validation.samples//batch_size,
                  callbacks=callbacks
                 )

<ipython-input-21-19d18e1a08c2>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  cnn.fit_generator(train,validation_data=validation,epochs=1,


112/112 [==============================] - 828s 7s/step - loss: 0.6578 - accuracy: 0.8651 - val_loss: 0.6479 - val_accuracy: 0.8895
